<img height="60px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="20px" vspace="5px">

# Sentiment Analysis on Imdb Movie Reviews

* References:

[https://towardsdatascience.com/how-to-build-a-neural-network-with-keras-e8faa33d0ae4](https://towardsdatascience.com/how-to-build-a-neural-network-with-keras-e8faa33d0ae4)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-Playbook/blob/master/NLP/Sentiment%20Analysis%20on%20Imdb/Neural%20Network%20with%20Keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-Playbook/blob/master/NLP/Sentiment%20Analysis%20on%20Imdb/Neural%20Network%20with%20Keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

##Import Dependencies and get the Data

In [0]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers



Using TensorFlow backend.


In [0]:
#The imdb sentiment classification dataset consists of 50,000 movie reviews from imdb users that are labeled as either positive (1) or negative (0). 
#The reviews are preprocessed and each one is encoded as a sequence of word indexes in the form of integers. 
#The words within the reviews are indexed by their overall frequency within the dataset. For example, the integer “2” encodes the second most frequent word in the data. 
#The 50,000 reviews are split into 25,000 for training and 25,000 for testing. 
from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)

#We don’t want to have a 50/50 train test split, we merge the data into data and targets after downloading, so that we can do an 80/20 split later on.
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)


17465344/17464789 [==============================] - 3s 0us/step


##Exploring the Data

In [0]:
print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))

length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))
print("Label:", targets[0])

#First review of the dataset which is labeled as positive (1). 
print(data[0])

#The code below retrieves the dictionary mapping word indices back into the original words so that we can read them. 
#It replaces every unknown word with a “#”. It does this by using the get_word_index() function.
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[0]] )
print(decoded) 

Categories: [0 1]
Number of unique words: 9998
Average Review length: 234.75892
Standard Deviation: 173.0
Label: 1
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283,

##Data Preparation

In [0]:
#We will vectorize every review and fill it with zeros so that it contains exactly 10,000 numbers. That means we fill every review that is shorter than 10,000 with zeros. 
#We do this because the biggest review is nearly that long and every input for our neural network needs to have the same size.
#We also transform the targets into floats.
def vectorize(sequences, dimension = 10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1
  return results
 
data = vectorize(data)
targets = np.array(targets).astype("float32")

In [0]:
#Now we split our data into a training and a testing set. The training set will contain 40,000 reviews and the testing set 10,000.
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]


##Building and Training the Model

In [0]:
model = models.Sequential()
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                500050    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 505,201
Trainable params: 505,201
Non-trainable params: 0
_________________________________________________________________


## compiling the model

In [0]:
#We use the “adam” optimizer. The optimizer is the algorithm that changes the weights and biases during training. 
#We also choose binary-crossentropy as loss (because we deal with binary classification) and accuracy as our evaluation metric.
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)


##Train our model

In [0]:
#We do this with a batch_size of 500 and only for two epochs because I recognized that the model overfits if we train it longer. 
#The Batch size defines the number of samples that will be propagated through the network and an epoch is an iteration over the entire training data. 
#In general a larger batch-size results in faster training, but don’t always converges fast. 
#A smaller batch-size is slower in training but it can converge faster. This is definitely problem dependent and you need to try out a few different values. 
#If you start with a problem for the first time, I would you recommend to you to first use a batch-size of 32, which is the standard size.
results = model.fit(
 train_x, train_y,
 epochs= 2,
 batch_size = 500,
 validation_data = (test_x, test_y)
)


Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 5s 125us/step - loss: 0.4061 - acc: 0.8198 - val_loss: 0.2641 - val_acc: 0.8935
Epoch 2/2
40000/40000 [==============================] - 3s 86us/step - loss: 0.2120 - acc: 0.9203 - val_loss: 0.2608 - val_acc: 0.8955


##Evaluate our model

In [0]:
print("Test-Accuracy:", np.mean(results.history["val_acc"]))

Test-Accuracy: 0.8945000082254411
